<a href="https://colab.research.google.com/github/nagi1995/Architectural-Heritage-Elements-image-Dataset/blob/main/ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import os
import cv2
import pickle
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.activations import softmax


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

'Colab Notebooks'   images      yolov4_bus_car
 downloads	   'My Drive'   yolov4_Dog


In [ ]:
train_path = "/mydrive/images/train"
test_path = "/mydrive/images/test"

In [ ]:
def return_y_categorical(df):
  df.rename(columns = {0 : "y"}, inplace = True)
  df["y"] = df["y"].astype("category").cat.codes
  return df

# **color images**

---



In [ ]:
def return_x_y(path):
  x = []
  y = []
  for img_name in os.listdir(path):
    #print(img_name)
    img = image.load_img(path + "/" + img_name)
    img = image.img_to_array(img)
    x.append(img/255)
    y.append(img_name.split("_")[0])
  y = return_y_categorical(pd.DataFrame(y))
  return (np.array(x), np.array(y))

In [ ]:
x_train, y_train = return_x_y(train_path)
x_train.shape, y_train.shape

((6141, 128, 128, 3), (6141, 1))

In [ ]:
x_test, y_test = return_x_y(test_path)
x_test.shape, y_test.shape

((4094, 128, 128, 3), (4094, 1))

In [ ]:
model = []

In [ ]:
model = Sequential()
model.add(Flatten(input_shape = (128, 128, 3)))
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.25))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.25))
model.add(Dense(10, activation = softmax))


In [ ]:
model.compile(optimizer = "adam", loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 49152)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               25166336  
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1290      
Total params: 25,233,290
Trainable params: 25,233,290
Non-trainable params: 0
__________________________________________

In [ ]:
model.fit(x_train, y_train, epochs = 10)

Epoch 1/10


ResourceExhaustedError: ignored

In [ ]:
model.save("/mydrive/ann_color.h5")

In [ ]:
model.evaluate(x_test, y_test)[1]*100





# **black and white images**

---



In [ ]:
def return_x_y_black_and_white(path):
  x = []
  y = []
  for img_name in os.listdir(path):
    #print(img_name)
    img = cv2.imread(path + "/" + img_name, 0)
    x.append(img/255)
    y.append(get_label_in_integer(img_name.split("_")[0]))
  y = return_y_categorical(pd.DataFrame(y))
  return (np.array(x), np.array(y))

In [ ]:
x_train, y_train = return_x_y_black_and_white(train_path)
x_train.shape, y_train.shape

((6141, 128, 128), (6141, 1))

In [ ]:
x_test, y_test = return_x_y_black_and_white(test_path)
x_test.shape, y_test.shape

((4094, 128, 128), (4094, 1))

In [ ]:
model = Sequential()
model.add(Flatten(input_shape = (128, 128)))
model.add(Dense(4096, activation = "relu"))
model.add(Dropout(0.25))
model.add(Dense(2048, activation = "relu"))
model.add(Dropout(0.25))
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.25))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.25))
model.add(Dense(10, activation = softmax))


ResourceExhaustedError: ignored

In [ ]:
model.compile(optimizer = "adam", loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs = 10)

In [ ]:
model.save("/mydrive/ann_black_and_white.h5")

In [ ]:
model.evaluate(x_test, y_test)[1]*100